In [38]:
import random
from copy import deepcopy
import pickle
import pandas
import numpy as np 
import itertools 
np.random.seed(0)

In [39]:
def draw_block(combo_dict):
    """
    drawing set of 90 locations for trials
    """
    trials = []

    cross_dict = {
        80: random.choice([6, 7]),
        40: 2,
        20 : random.choice([0, 1]),
        10: 0,
        0: 0,
        -10: 0,
        -40: 2,
    }
    cross_dict[-80] = 13 - cross_dict[80]
    cross_dict[-20] = 1 - cross_dict[20]

    azim_deltas = [-80, -40, -20, -10, 0, 10, 20, 40, 80]
    elev_deltas = [-40, -10, 0, 10, 40]
    sex = ['same', 'opposite']

    dict_copy = deepcopy(combo_dict)
    for a_delta in azim_deltas:
        for e_delta in elev_deltas:
            for s in sex:
                cross = False
                if cross_dict[a_delta]:
                    cross = True
                    cross_dict[a_delta] -= 1

                if cross:
                    azims = dict_copy[f'azim_{a_delta}_crossed'].pop(random.randrange(len(dict_copy[f'azim_{a_delta}_crossed'])))
                else:
                    azims = dict_copy[f'azim_{a_delta}_within'].pop(random.randrange(len(dict_copy[f'azim_{a_delta}_within'])))

                elevs = random.sample(dict_copy[f'elev_{e_delta}'], 1)[0]
                trials.append(((azims[0], elevs[0]), (azims[1], elevs[1]), s, cross, a_delta, e_delta))
    return trials

In [46]:
def draw_block_0_azim():
    target_locs = [-90, -30, 0, 30, 90]
    distractor_locs = [-90, -60, -30, 0, 30, 60, 90]
    sex_assignments = np.random.choice(['same', 'opposite'], size=len(target_locs) * len(distractor_locs))
    crossed = lambda x: True if x >=0 else False
    # return tuples of (target_loc, distractor_loc, same/diff, same-hemifield, azim_delta, elev_delta)
    trials = [((targ_azim, 0), (dist_azim, 0), sex_assignments[ix], crossed(targ_azim*dist_azim), dist_azim - targ_azim, 0)
              for ix, (targ_azim, dist_azim) in enumerate(itertools.product(*[target_locs, distractor_locs]))
                ]
    return trials

In [53]:
def draw_block_elev(azims=[-90, -30, 0, 30, 90], 
                    target_elevs = [-20, 10, 40],
                    distractor_elevs = [-20, -10, 0, 10, 20, 30, 40]
):
    sex_assignments =  np.random.choice(['same', 'opposite'], size=len(azims) * len(target_elevs) * len(distractor_elevs))
    # return tuples of (target_loc, distractor_loc, same/diff, same-hemifield, azim_delta, elev)
    trials = [((azim, target_elev), (azim, distractor_elev), sex_assignments[ix], 0, 0, distractor_elev - target_elev)
                for ix, (azim, target_elev, distractor_elev) in enumerate(itertools.product(*[azims, target_elevs, distractor_elevs]))
                ]
    return trials

## Get trials locally

In [54]:
manifest = pandas.read_pickle('full_eval_trial_manifest_new_fnames_local_fnames.pdpkl')

In [55]:
def create_experiment(manifest, num_blocks=4):
    """
    create experiment with 4 blocks
    """
    experiment = []
    azim_elev_combos = pickle.load(open('azim_elev_combos.pkl', 'rb'))
    for i in range(num_blocks):
        block = draw_block(azim_elev_combos)
        random.shuffle(block)
        experiment.append(block)

    words = list(manifest.word.unique())
    random.shuffle(words)

    same_gender = 'male_male'
    opp_gender = 'male_female'
    i = 0
    array_manifest = []
    for block in experiment:
        for j, trial in enumerate(block):
            trial_word = words[i]
            i += 1

            if trial[2] == 'same':
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{same_gender}')].index[0]
                true_cond = same_gender
                if same_gender == 'male_male':
                    same_gender = 'female_female'
                else:
                    same_gender = 'male_male'

            else:
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{opp_gender}')].index[0]
                true_cond = opp_gender
                if opp_gender == 'male_female':
                    opp_gender = 'female_male'
                else:
                    opp_gender = 'male_female'
            trial_row = manifest.iloc[manifest_idx]
            cue_src_fn = trial_row['cue_src_fn']
            target_src_fn = trial_row['src_fn']
            distractor_src_fn = trial_row['distractor_src_fn']
            distractor_word = trial_row['distractor_word']
            block[j] = block[j] + (trial_word, true_cond, manifest_idx, distractor_word)
            array_manifest.append((trial[0], trial[1], cue_src_fn, target_src_fn, distractor_src_fn))

    experiment_data = dict()
    for i, block in enumerate(experiment):
        experiment_data[f'block_{i}'] = dict()
        block_dict = experiment_data[f'block_{i}']
        for j, trial in enumerate(block):
            block_dict[f'trial_{j}'] = {'target_loc': trial[0],
                                        'distractor_loc': trial[1],
                                        'sex_cond': trial[7],
                                        'crossed': trial[3],
                                        'azim_delta': trial[4],
                                        'elev_delta': trial[5],
                                        'target_word': trial[6],
                                        'distractor_word': trial[9]}

    return experiment_data, array_manifest

In [56]:
def create_new_experiment(manifest, num_azim_blocks=4, num_elev_blocks=2):
    """
    create experiment with two types of blocks
    """
    experiment = []
    for i in range(num_azim_blocks):
        block = draw_block_0_azim()
        random.shuffle(block)
        experiment.append(block)

    for i in range(num_elev_blocks):
        block = draw_block_elev()
        random.shuffle(block)
        experiment.append(block)

    words = list(manifest.word.unique())
    random.shuffle(words)

    same_gender = 'male_male'
    opp_gender = 'male_female'
    i = 0
    array_manifest = []
    for block in experiment:
        for j, trial in enumerate(block):
            try:
                trial_word = words[i]
            except:
                i = 0
                trial_word = words[i]
            i += 1

            if trial[2] == 'same':
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{same_gender}')].index[0]
                true_cond = same_gender
                if same_gender == 'male_male':
                    same_gender = 'female_female'
                else:
                    same_gender = 'male_male'

            else:
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{opp_gender}')].index[0]
                true_cond = opp_gender
                if opp_gender == 'male_female':
                    opp_gender = 'female_male'
                else:
                    opp_gender = 'male_female'
            trial_row = manifest.iloc[manifest_idx]
            cue_src_fn = trial_row['cue_src_fn']
            target_src_fn = trial_row['src_fn']
            distractor_src_fn = trial_row['distractor_src_fn']
            distractor_word = trial_row['distractor_word']
            block[j] = block[j] + (trial_word, true_cond, manifest_idx, distractor_word)
            array_manifest.append((trial[0], trial[1], cue_src_fn, target_src_fn, distractor_src_fn))

    experiment_data = dict()
    for i, block in enumerate(experiment):
        experiment_data[f'block_{i}'] = dict()
        block_dict = experiment_data[f'block_{i}']
        for j, trial in enumerate(block):
            block_dict[f'trial_{j}'] = {'target_loc': trial[0],
                                        'distractor_loc': trial[1],
                                        'sex_cond': trial[7],
                                        'crossed': trial[3],
                                        'azim_delta': trial[4],
                                        'elev_delta': trial[5],
                                        'target_word': trial[6],
                                        'distractor_word': trial[9]}
    trial_dict = {i:vals for i,vals in enumerate(array_manifest)}
    return experiment_data, array_manifest, trial_dict

In [61]:
# write out manifests 
import pickle 
from pathlib import Path
# get n files in output dir 
out_dir = Path('speaker_array_manifests/pilot_v02')
out_dir.mkdir(exist_ok=True, parents=True)
n_files = len(list(out_dir.glob('*manifest.pkl')))

np.random.seed(n_files) # change seed for each participant!!!! 
experiment, array_manifest, trial_dict = create_new_experiment(manifest, num_azim_blocks=3, num_elev_blocks=1)
print([len(experiment[key]) for key in experiment.keys()])

part_ix = f"participant_{n_files+1:03d}"
print(part_ix)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_meta.pkl', 'wb') as f:
    pickle.dump(experiment, f)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_array_manifest.pkl', 'wb') as f:
    pickle.dump(array_manifest, f)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_trial_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict, f)
    
# get target key list 
out_path = Path("expmt_keys")

word_key = [trial['target_word'] for block in experiment.values() for trial in block.values()]

# save as json 
import json 
with open(out_path / f"{part_ix}_key.json", "w") as f:
    json.dump(word_key, f)



[35, 35, 35, 105]
participant_001
